In [ ]:
import codecs
import os
import sys
import threading
import serial
from serial.tools.list_ports import comports

1. Gimbal control

In [ ]:
from a2gmeasurements import GimbalRS2
import time, timeit
import numpy as np

myGimbal = GimbalRS2()
myGimbal.start_thread_gimbal()

In [ ]:
from a2gmeasurements import GimbalRS2
import time, timeit
import numpy as np

myGimbal = GimbalRS2()
myGimbal.start_thread_gimbal()

assumed_time_for_execute_time = 0.0015 # s
gimbal_speed = (1800/3.47) # DEG/s
condition_continue = True
tol = gimbal_speed * assumed_time_for_execute_time
cnt = 0
myGimbal.request_current_position()
time.sleep(0.0015)
last_yaw = myGimbal.yaw 
last_roll = myGimbal.roll
print('Outside: ',last_yaw, last_roll)
myGimbal.setPosControl(yaw=1250, roll=0, pitch=0)
time.sleep(1)
while(condition_continue):
    print('Execution time: ', timeit.default_timer() - start)
    myGimbal.request_current_position()
    time.sleep(0.0015)
    print('Inside: ', myGimbal.yaw, myGimbal.roll)
    
    if np.abs(last_yaw - myGimbal.yaw) < tol and np.abs(last_roll - myGimbal.roll) < tol:
        condition_continue = False
    else:
        last_yaw = myGimbal.yaw
        last_roll = myGimbal.roll
        start = timeit.default_timer()
    cnt = cnt+1
    print(cnt)
    


In [ ]:
#myGimbal.request_current_position()
myGimbal.setPosControl(yaw=0, roll=0, pitch=0)

In [ ]:
myGimbal.stop_thread_gimbal()
time.sleep(0.05)
myGimbal.actual_bus.shutdown()

2. Anritsu Control

In [ ]:
from a2gmeasurements import myAnritsuSpectrumAnalyzer

mySA = myAnritsuSpectrumAnalyzer()
mySA.spectrum_analyzer_connect()
rx_power =  mySA.retrieve_max_pow()
print(rx_power)
mySA.spectrum_analyzer_close()

3. Septentrio Control

In [ ]:
from a2gmeasurements import GpsSignaling
mySeptentrioGPS = GpsSignaling(DBG_LVL_2=True, DBG_LVL_0=True)

In [ ]:
mySeptentrioGPS.serial_connect()
mySeptentrioGPS.start_thread_gps()
mySeptentrioGPS.start_gps_data_retrieval(msg_type='NMEA', nmea_type='GGA', interval='sec1') # HDF instead of GGA for heading info
#mySeptentrioGPS.start_gps_data_retrieval(msg_type='SBF')

In [ ]:
mySeptentrioGPS.stop_gps_data_retrieval(msg_type='NMEA')
mySeptentrioGPS.stop_thread_gps()

In [ ]:
print(len(mySeptentrioGPS.SBF_frame_buffer), len(mySeptentrioGPS.NMEA_buffer))
if len(mySeptentrioGPS.SBF_frame_buffer) > 0:
    print(mySeptentrioGPS.SBF_frame_buffer[-1])
if len(mySeptentrioGPS.NMEA_buffer) > 0:
    print(mySeptentrioGPS.NMEA_buffer[-1])

Precision of distance when using planar coordinates ('precision' is given by google estimate of distance)

In [ ]:
import pyproj as proj
from a2gmeasurements import HelperA2GMeasurements
import numpy as np

pt_hammentie_50 = [60.186977663598164, 24.961182204780805]
pt_hakaniemi_mkt = [60.17907971164074, 24.951228689442726]

h = HelperA2GMeasurements()

lat_pln_hamm, lon_pln_hamm = h.convert_DDMMS_to_planar(24.961182204780805, 60.186977663598164)
lat_pln_hak, lon_pln_hak = h.convert_DDMMS_to_planar(24.951228689442726, 60.17907971164074)

# Compare this distance with Google MAPS: CHECKED ---> within 2m error
d_hamm_hak = np.linalg.norm(np.array([lon_pln_hamm, lat_pln_hamm]) - np.array([lon_pln_hak, lat_pln_hak]))

Example of a real GPS NMEA output

In [20]:
real_gps_data = {'Timestamp': '110815.00', 
                 'Latitude': '6011.1011606', 
                 'Latitude Direction': 'N', 
                 'Longitude': '02448.9144475', 
                 'Longitude Direction': 'E', 
                 'GPS Quality Indicator': '2', 
                 'Number of Satellites in use': '28', 
                 'Horizontal Dilution of Precision': '0.6', 
                 'Antenna Alt above sea level (mean)': '0.9230', 
                 'Units of altitude (meters)': 'M', 
                 'Geoidal Separation': '19.5883', 
                 'Units of Geoidal Separation (meters)': 'M', 
                 'Age of Differential GPS Data (secs)': '1.8', 
                 'Differential Reference Station ID': '0123'}

Server connection

In [ ]:
from socket import socket
import time
from a2gmeasurements import HelperA2GMeasurements

host = 'localhost'
myHelper = HelperA2GMeasurements('GROUND', host)
myHelper.HelperStartA2GCom()


In [ ]:
myHelper.HelperA2GStopCom()
print(myHelper.SOCKET_BUFFER, len(myHelper.SOCKET_BUFFER))

5. Signal generator

In [ ]:
import pyvisa

rm = pyvisa.ResourceManager()
print(rm.list_resources())


In [ ]:
inst = rm.open_resource('GPIB0::19::INSTR')
print(inst.query("*IDN?"))


In [ ]:
inst.write('F0 28.95 GH\n')
inst.write('L0 20 DM\n')

In [ ]:
# Turn On RF output
inst.write('RF1\n')

In [ ]:
# Turn Off RF output
inst.write('RF0\n')

Example of code for Measurements

In [ ]:
import json
import numpy as np
from socket import socket
import time
from a2gmeasurements import HelperA2GMeasurements, GimbalRS2, GpsSignaling

host = ''
myHelper = HelperA2GMeasurements('GROUND', host, DBG_LVL_0=False, 
                                 DBG_LVL_1=True, 
                                 IsGPS=True, 
                                 IsGimbal=False, 
                                 IsSignalGenerator=False,
                                 F0=28.95,
                                 L0=-30)

In [ ]:
myHelper.myGimbal.request_current_position()
time.sleep(0.002)

# Gimbal assumed to be not moving
yaw_now = myHelper.myGimbal.yaw
roll_now = myHelper.myGimbal.roll
print('Compare yaw printed value previously, with yaw_now var: ', yaw_now, '\n')
print('Compare roll printed value previously, with roll_now var: ', roll_now, '\n')

while(myHelper.mySeptentrioGPS.NMEA_buffer == []):
    1
    
lat_ground = float(myHelper.mySeptentrioGPS.NMEA_buffer[-1]['Latitude'])/10
lon_ground = float(myHelper.mySeptentrioGPS.NMEA_buffer[-1]['Longitude'])/10
height_ground = float(myHelper.mySeptentrioGPS.NMEA_buffer[-1]['Antenna Alt above sea level (mean)'])
print('This is are the coordinates for the GROUND station\n')
print(f'LAT:{lat_ground}, LON: {lon_ground}, HEIGHT: {height_ground}')

# Point GROUND gimbal towards DRONE
lat_drone = 60.1857
lon_drone = 24.81465
height_drone = 30

yaw, roll = myHelper.mobile_gimbal_follows_drone(yaw_now, lat_ground, lon_ground, height_ground, 
                                                lat_drone, lon_drone, height_drone)

myHelper.myGimbal.setPosControl(yaw, roll, 0)

Move the gimbal

In [ ]:
# 9. Request Gimbal orientation
meas_number=1
myHelper.myGimbal.request_current_position()
time.sleep(0.0015)

# Gimbal assumed to be not moving
yaw_now = myHelper.myGimbal.yaw
roll_now = myHelper.myGimbal.roll
print('Compare yaw printed value previously, with yaw_now var: ', yaw_now, '\n')
print('Compare yaw printed value previously, with roll_now var: ', roll_now, '\n')

# 10. Move GROUND gimbal in 30 deg steps
N_ang_steps = 360/120
reset_yaw_gimbal = myHelper.send_N_azimuth_angles(int(yaw_now*10), int(roll_now*10), int(N_ang_steps), meas_number=meas_number)

# 11. Make the gimbal move in the opposite direction to unfold cables
for i in reset_yaw_gimbal:
    myHelper.myGimbal.setPosControl(i, int(roll_now*10), 0)
    time.sleep(3.5)

#input('Stop test?')
myHelper.HelperA2GStopCom()

In [ ]:
lat_gimbal = float(myHelper.mySeptentrioGPS.NMEA_buffer[-1]['Latitude'][0:2]) + float(myHelper.mySeptentrioGPS.NMEA_buffer[-1]['Latitude'][2:])/60
lon_gimbal = float(myHelper.mySeptentrioGPS.NMEA_buffer[-1]['Longitude'][0:3]) + float(myHelper.mySeptentrioGPS.NMEA_buffer[-1]['Longitude'][3:])/60
height_gimbal = float(myHelper.mySeptentrioGPS.NMEA_buffer[-1]['Antenna Alt above sea level (mean)'])

print('This is are the coordinates for the GROUND station\n')
print(f'LAT:{lat_gimbal}, LON: {lon_gimbal}, HEIGHT: {height_gimbal}')

# Point GROUND gimbal towards DRONE
lat_drone = 60.15893
lon_drone = 24.91091
height_drone = 40

lat_drone_planar, lon_drone_planar = myHelper.convert_DDMMS_to_planar(lon_drone, lat_drone, offset=None, epsg_in=4326, epsg_out=3901)
lat_gimbal_planar, lon_gimbal_planar = myHelper.convert_DDMMS_to_planar(lon_gimbal, lat_gimbal, offset=None, epsg_in=4326, epsg_out=3901)
        
position_drone = np.array([lon_drone_planar, lat_drone_planar, height_drone])
position_gimbal = np.array([lon_gimbal_planar, lat_gimbal_planar, height_gimbal])
print(f"[DEBUG]: POS_DRONE: {position_drone}, POS_GROUND: {position_gimbal}")
        
#d_mobile_drone_3D = np.linalg.norm(position_drone - position_gimbal)
d_mobile_drone_2D = np.linalg.norm(position_drone[:-1] - position_gimbal[:-1])
print(f"[DEBUG]: Azimuth distance from ground to drone station: {d_mobile_drone_2D}")
                
# Elevation angle of the drone referenced to GROUND station plane. 
# Is also the roll angle to send to the gimbal
roll_to_set = np.arctan2(height_drone - height_gimbal, d_mobile_drone_2D)
roll_to_set = int(np.rad2deg(roll_to_set)*10)

print(roll_to_set)